In [1]:
!pip install html5lib beautifulsoup4
!pip install selenium

from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import requests
import json
import re
import pandas as pd

In [2]:
from urllib import request
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException

In [3]:
from selenium.webdriver.common.keys import Keys

In [4]:
import numpy as np

In [5]:
# 인코딩이 euc-kr입니다. 저걸 지정하지 않으면 csv가 안 열려요!
# idol_id가 저장된 csv 파일을 불러오셔야 합니다. 멜론 url에 가수를 찾기 위한 id입니다. 

idol_list = pd.read_csv('1996_1997_debut.csv', encoding='euc-kr')
idol_list.head()

,artist,artist_eng,member_num,leader_name,gender,agency,idol_id
0,언타이틀,Untitle,2,유건형,남,예당 음향 (예당 컴퍼니),100042
1,Hamohamo,HAMO HAMO,2,이성진,남,뮤직 팩토리,100178
2,아이돌,IDOL,2,최혁준,남,DSP 미디어,101640
3,구피,Goofy,3,이승광,남,NaN,100023
4,태사자,Taesaja,4,김형준,남,매스컴,100096


In [6]:
idol_list.shape

(10, 7)

In [7]:
idol_list = idol_list[np.isfinite(idol_list['idol_id'])].reset_index(drop=True)

In [8]:
idol_list.shape

(10, 7)

In [9]:
# id가 flaot으로 저장되어 있습니다. int로 바꿔주고 저장하겠습니다. 
idol_list['idol_id'] = idol_list['idol_id'].astype(int)
idol_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   artist       10 non-null     object
 1   artist_eng   10 non-null     object
 2   member_num   10 non-null     object
 3   leader_name  9 non-null      object
 4   gender       10 non-null     object
 5   agency       8 non-null      object
 6   idol_id      10 non-null     int64 
dtypes: int64(1), object(6)
memory usage: 688.0+ bytes


In [10]:
# 수집을 위한 빈 리스트 생성

title = []
release_date = []
song_genre = []
lyrics = []
creator = []
artist = []
artist_m = []
debut_y = []
debut_t = []
agency = []

In [11]:
# 곡마다 가수 이름을 붙여주어야 하기에 가수 이름 리스트로 받아오기
names = idol_list.artist
names[0]

'언타이틀'

In [12]:
# 크롤러가 링크를 찾을 때 매번 바꿔주는 값. for문에 사용됩니다. 
ids = idol_list.idol_id
ids[0]

100042

In [13]:
# tests = ['780066']

## 크롤러

In [14]:
path = "/Users/mac/opt/anaconda3/bin/chromedriver"
driver = webdriver.Chrome(path)
driver.get("https://www.melon.com")

In [15]:
# 곡 정보를 찾아들어가기 위한 url을 분해해서 변수 지정해놓은 모습입니다. 
# l3 뒤에 붙는 숫자가 멜론 서버에 저장된 해당 가수의 해당 곡 번호입니다. 

l1 = 'https://www.melon.com/artist/song.htm?artistId='
# idol_id
l2 = '#params%5BlistType%5D=A&params%5BorderBy%5D=ISSUE_DATE&params%5BartistId%5D='
# idol_id
l3 = '&po=pageObj&startIndex='
# l1 + 'idol_id' + l2 + 'idol_id' + l3 + number

# 가수 이름을 넣기 위한 변수
i = 0
tmp = []
str_tmp = ""
arr = []

for id in ids:
    
    n = 1
    list_n = 1
    while True:
    #for k in range(1):
                 
        
        print('n: ',n)
        driver.get(l1+str(id)+l2+str(id)+l3+str(n))    
        sleep(2)
        
                
        # 곡 정보로 들어가기
        try:
            driver.find_element_by_xpath('//*[@id="conts"]/div[3]/ul/li[3]/a').click()
            str_tmp = '//*[@id="frm"]/div/table/tbody/tr[' + str(list_n) + ']/td[3]/div/div/a[1]'
            driver.find_element_by_xpath(str_tmp).click()
            print('list: ',list_n)
            # 발매날짜 크롤링
            try:
                str_tmp = driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[2]').text
                arr = str_tmp.split('.')
                print("check :",str_tmp)
                if( int(arr[0]) < 2000 and int(arr[0]) > 1995): # 96,97,98,99 발매 노래만
                    print("input: ", str_tmp)
                    release_date.append(str_tmp)
                    list_n += 1
                elif (int(arr[0]) < 1996): # 96년 이전 발매일 경우 break
                    break
                else:
                    list_n += 1
                    continue
                
            
            except NoSuchElementException as e:
                release_date.append('')
            
            # 제목 크롤링(제목은 없지 않기 때문에 예외 없음)
            title.append(driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[1]/div[1]').text)
            print('title:', title)

            try:
                song_genre.append(driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[3]').text)
            except NoSuchElementException as e:
                song_genre.append('')

            # 가사 크롤링
            try:
                lyrics.append(driver.find_element_by_xpath('//*[@id="d_video_summary"]').text)
            except NoSuchElementException as e:
                lyrics.append('')

            # 작사 작곡 내역이 없는 경우
            try:
                creator.append(driver.find_element_by_xpath('//*[@id="conts"]/div[3]/ul').text)
            except NoSuchElementException as e:
                creator.append('')

            n += 1
            
            # 가수 이름 붙이기
            artist.append(names[i])
            
            # 디버깅    
            print(len(title), title[len(title)-1])

        except NoSuchElementException as e:
            i += 1
            break
        
        

n:  1
list:  1
check : 2007.12.24
n:  1
list:  2
check : 2007.12.13
n:  1
list:  3
check : 2003.11.01
n:  1
list:  4
check : 2000.11.01
n:  1
list:  5
check : 1999.08.01
input:  1999.08.01
title: ['떠나가지 마세요']
1 떠나가지 마세요
n:  2
list:  6
check : 1999.03.05
input:  1999.03.05
title: ['떠나가지 마세요', '학교 (Main Title)']
2 학교 (Main Title)
n:  3
list:  7
check : 1999.02.01
input:  1999.02.01
title: ['떠나가지 마세요', '학교 (Main Title)', '떠나가지 마세요']
3 떠나가지 마세요
n:  4
list:  8
check : 1999.02.01
input:  1999.02.01
title: ['떠나가지 마세요', '학교 (Main Title)', '떠나가지 마세요', '나의 슬픈 사랑이야기']
4 나의 슬픈 사랑이야기
n:  5
list:  9
check : 1999.02.01
input:  1999.02.01
title: ['떠나가지 마세요', '학교 (Main Title)', '떠나가지 마세요', '나의 슬픈 사랑이야기', '사랑연가']
5 사랑연가
n:  6
list:  10
check : 1999.02.01
input:  1999.02.01
title: ['떠나가지 마세요', '학교 (Main Title)', '떠나가지 마세요', '나의 슬픈 사랑이야기', '사랑연가', '댄스! 댄스! 댄스!']
6 댄스! 댄스! 댄스!
n:  7
list:  11
check : 1999.02.01
input:  1999.02.01
title: ['떠나가지 마세요', '학교 (Main Title)', '떠나가지 마세요', '나의 슬픈 사랑이야기', '사랑연가', '댄스

list:  37
check : 1997.11.01
input:  1997.11.01
title: ['떠나가지 마세요', '학교 (Main Title)', '떠나가지 마세요', '나의 슬픈 사랑이야기', '사랑연가', '댄스! 댄스! 댄스!', '하얀 사랑', '낮과 밤', '인연', '검은 위선', 'Love Song #2', 'Run', 'Play Boy', 'Goodbye', '우리가 시작할때...거기엔 랩이 있었다', 'Intro', '꽃', '승리', '눈 내리는 날', 'Don`t Stop The Music', '불', '정.경', '왜', '빈자리', '자살', '물거품', 'Disco', '앞으로', '꽃 (Remix)', 'Outro', '고백 (New Song)', '오늘밤 (Late Night Mix)', '증오 (Hardcore Techno Mix)']
33 증오 (Hardcore Techno Mix)
n:  34
list:  38
check : 1997.11.01
input:  1997.11.01
title: ['떠나가지 마세요', '학교 (Main Title)', '떠나가지 마세요', '나의 슬픈 사랑이야기', '사랑연가', '댄스! 댄스! 댄스!', '하얀 사랑', '낮과 밤', '인연', '검은 위선', 'Love Song #2', 'Run', 'Play Boy', 'Goodbye', '우리가 시작할때...거기엔 랩이 있었다', 'Intro', '꽃', '승리', '눈 내리는 날', 'Don`t Stop The Music', '불', '정.경', '왜', '빈자리', '자살', '물거품', 'Disco', '앞으로', '꽃 (Remix)', 'Outro', '고백 (New Song)', '오늘밤 (Late Night Mix)', '증오 (Hardcore Techno Mix)', '첫사랑 (Acoustic Hiphop Mix)']
34 첫사랑 (Acoustic Hiphop Mix)
n:  35
list:  39
check : 1997

list:  7
check : 1996.11
input:  1996.11
title: ['떠나가지 마세요', '학교 (Main Title)', '떠나가지 마세요', '나의 슬픈 사랑이야기', '사랑연가', '댄스! 댄스! 댄스!', '하얀 사랑', '낮과 밤', '인연', '검은 위선', 'Love Song #2', 'Run', 'Play Boy', 'Goodbye', '우리가 시작할때...거기엔 랩이 있었다', 'Intro', '꽃', '승리', '눈 내리는 날', 'Don`t Stop The Music', '불', '정.경', '왜', '빈자리', '자살', '물거품', 'Disco', '앞으로', '꽃 (Remix)', 'Outro', '고백 (New Song)', '오늘밤 (Late Night Mix)', '증오 (Hardcore Techno Mix)', '첫사랑 (Acoustic Hiphop Mix)', '돕자 (Krush Groove Mix)', '날개 (Super Funky Mix)', '등대지기 (New Song)', '책임져 (Boombox Mix)', '파란색 (Untitle`s Rock Mix)', '부 (Don`t Stop Mix)', 'Pang Pang', '빠삐용 (그의 두통은 자유였다)', '마라! 봐라!', '아니벌써', 'Silver Line', 'True Love', '반쪽만의 사랑 (My Teacher)']
47 반쪽만의 사랑 (My Teacher)
n:  8
list:  8
check : 1996.11
input:  1996.11
title: ['떠나가지 마세요', '학교 (Main Title)', '떠나가지 마세요', '나의 슬픈 사랑이야기', '사랑연가', '댄스! 댄스! 댄스!', '하얀 사랑', '낮과 밤', '인연', '검은 위선', 'Love Song #2', 'Run', 'Play Boy', 'Goodbye', '우리가 시작할때...거기엔 랩이 있었다', 'Intro', '꽃', '승리', '눈 내리는 날', '

list:  7
check : 1996.11
input:  1996.11
title: ['떠나가지 마세요', '학교 (Main Title)', '떠나가지 마세요', '나의 슬픈 사랑이야기', '사랑연가', '댄스! 댄스! 댄스!', '하얀 사랑', '낮과 밤', '인연', '검은 위선', 'Love Song #2', 'Run', 'Play Boy', 'Goodbye', '우리가 시작할때...거기엔 랩이 있었다', 'Intro', '꽃', '승리', '눈 내리는 날', 'Don`t Stop The Music', '불', '정.경', '왜', '빈자리', '자살', '물거품', 'Disco', '앞으로', '꽃 (Remix)', 'Outro', '고백 (New Song)', '오늘밤 (Late Night Mix)', '증오 (Hardcore Techno Mix)', '첫사랑 (Acoustic Hiphop Mix)', '돕자 (Krush Groove Mix)', '날개 (Super Funky Mix)', '등대지기 (New Song)', '책임져 (Boombox Mix)', '파란색 (Untitle`s Rock Mix)', '부 (Don`t Stop Mix)', 'Pang Pang', '빠삐용 (그의 두통은 자유였다)', '마라! 봐라!', '아니벌써', 'Silver Line', 'True Love', '반쪽만의 사랑 (My Teacher)', '히든 카드', '세상에 던지는 한마디', 'Pang Pang (나보다 나은 너)', '빠삐용 (그의 두통은 자유였다) (Hamo Ver.)', 'Intro', 'Be My Guest', '환상체험', '건망증', '이럴수가', 'Love Story', '통화중 (감기)']
58 통화중 (감기)
n:  8
list:  8
check : 1996.11
input:  1996.11
title: ['떠나가지 마세요', '학교 (Main Title)', '떠나가지 마세요', '나의 슬픈 사랑이야기', '사랑연가', '댄스! 댄스!

list:  16
check : 1996.02
input:  1996.02
title: ['떠나가지 마세요', '학교 (Main Title)', '떠나가지 마세요', '나의 슬픈 사랑이야기', '사랑연가', '댄스! 댄스! 댄스!', '하얀 사랑', '낮과 밤', '인연', '검은 위선', 'Love Song #2', 'Run', 'Play Boy', 'Goodbye', '우리가 시작할때...거기엔 랩이 있었다', 'Intro', '꽃', '승리', '눈 내리는 날', 'Don`t Stop The Music', '불', '정.경', '왜', '빈자리', '자살', '물거품', 'Disco', '앞으로', '꽃 (Remix)', 'Outro', '고백 (New Song)', '오늘밤 (Late Night Mix)', '증오 (Hardcore Techno Mix)', '첫사랑 (Acoustic Hiphop Mix)', '돕자 (Krush Groove Mix)', '날개 (Super Funky Mix)', '등대지기 (New Song)', '책임져 (Boombox Mix)', '파란색 (Untitle`s Rock Mix)', '부 (Don`t Stop Mix)', 'Pang Pang', '빠삐용 (그의 두통은 자유였다)', '마라! 봐라!', '아니벌써', 'Silver Line', 'True Love', '반쪽만의 사랑 (My Teacher)', '히든 카드', '세상에 던지는 한마디', 'Pang Pang (나보다 나은 너)', '빠삐용 (그의 두통은 자유였다) (Hamo Ver.)', 'Intro', 'Be My Guest', '환상체험', '건망증', '이럴수가', 'Love Story', '통화중 (감기)', '삼인사각', '어쨌건...', '도사 (Jungle Trance)', '귀여운 악마', 'Bow Wow', 'Home Boy', 'Street Boy', '둘만의 사랑', '꿈속의 그녀']
67 꿈속의 그녀
n:  17
list:  17
check 

list:  49
check : 1999.07
input:  1999.07
title: ['떠나가지 마세요', '학교 (Main Title)', '떠나가지 마세요', '나의 슬픈 사랑이야기', '사랑연가', '댄스! 댄스! 댄스!', '하얀 사랑', '낮과 밤', '인연', '검은 위선', 'Love Song #2', 'Run', 'Play Boy', 'Goodbye', '우리가 시작할때...거기엔 랩이 있었다', 'Intro', '꽃', '승리', '눈 내리는 날', 'Don`t Stop The Music', '불', '정.경', '왜', '빈자리', '자살', '물거품', 'Disco', '앞으로', '꽃 (Remix)', 'Outro', '고백 (New Song)', '오늘밤 (Late Night Mix)', '증오 (Hardcore Techno Mix)', '첫사랑 (Acoustic Hiphop Mix)', '돕자 (Krush Groove Mix)', '날개 (Super Funky Mix)', '등대지기 (New Song)', '책임져 (Boombox Mix)', '파란색 (Untitle`s Rock Mix)', '부 (Don`t Stop Mix)', 'Pang Pang', '빠삐용 (그의 두통은 자유였다)', '마라! 봐라!', '아니벌써', 'Silver Line', 'True Love', '반쪽만의 사랑 (My Teacher)', '히든 카드', '세상에 던지는 한마디', 'Pang Pang (나보다 나은 너)', '빠삐용 (그의 두통은 자유였다) (Hamo Ver.)', 'Intro', 'Be My Guest', '환상체험', '건망증', '이럴수가', 'Love Story', '통화중 (감기)', '삼인사각', '어쨌건...', '도사 (Jungle Trance)', '귀여운 악마', 'Bow Wow', 'Home Boy', 'Street Boy', '둘만의 사랑', '꿈속의 그녀', '노래 이야기', 'Na Na Na', '상상', 'Memo

list:  9
check : 1998.06
input:  1998.06
title: ['떠나가지 마세요', '학교 (Main Title)', '떠나가지 마세요', '나의 슬픈 사랑이야기', '사랑연가', '댄스! 댄스! 댄스!', '하얀 사랑', '낮과 밤', '인연', '검은 위선', 'Love Song #2', 'Run', 'Play Boy', 'Goodbye', '우리가 시작할때...거기엔 랩이 있었다', 'Intro', '꽃', '승리', '눈 내리는 날', 'Don`t Stop The Music', '불', '정.경', '왜', '빈자리', '자살', '물거품', 'Disco', '앞으로', '꽃 (Remix)', 'Outro', '고백 (New Song)', '오늘밤 (Late Night Mix)', '증오 (Hardcore Techno Mix)', '첫사랑 (Acoustic Hiphop Mix)', '돕자 (Krush Groove Mix)', '날개 (Super Funky Mix)', '등대지기 (New Song)', '책임져 (Boombox Mix)', '파란색 (Untitle`s Rock Mix)', '부 (Don`t Stop Mix)', 'Pang Pang', '빠삐용 (그의 두통은 자유였다)', '마라! 봐라!', '아니벌써', 'Silver Line', 'True Love', '반쪽만의 사랑 (My Teacher)', '히든 카드', '세상에 던지는 한마디', 'Pang Pang (나보다 나은 너)', '빠삐용 (그의 두통은 자유였다) (Hamo Ver.)', 'Intro', 'Be My Guest', '환상체험', '건망증', '이럴수가', 'Love Story', '통화중 (감기)', '삼인사각', '어쨌건...', '도사 (Jungle Trance)', '귀여운 악마', 'Bow Wow', 'Home Boy', 'Street Boy', '둘만의 사랑', '꿈속의 그녀', '노래 이야기', 'Na Na Na', '상상', 'Memor

## Dataframe에 크롤링된 데이터 저장하기

In [16]:
test_list = [title, release_date, song_genre, creator, lyrics]
column_names = ['title', 'artist', 'release_date', 'song_genre', 'creator', 'lyrics']
column_v = [title, artist, release_date, song_genre, creator, lyrics]

test_df = pd.DataFrame(column_v)


In [17]:
test_p = test_df.head(6).T
test_p.head()

,0,1,2,3,4,5
0,떠나가지 마세요,언타이틀,1999.08.01,댄스,,
1,학교 (Main Title),언타이틀,1999.03.05,-,,다시 바꿔 생각해 너무 억울한걸\n다른 위치라면은 누구나 같은데\n학교폭력에 숨진 ...
2,떠나가지 마세요,언타이틀,1999.02.01,댄스,유건형\n작사\n서정환\n작사\n유건형\n작곡\n유건형\n편곡,난 울고 있어요 그대도 아시나요\n미치도록 아픈 나의 가슴을\n그대 날 떠난 후엔 ...
3,나의 슬픈 사랑이야기,언타이틀,1999.02.01,댄스,유건형\n작사\n유건형\n작곡\n유건형\n편곡,이젠 늦었어요 이미 그녀는\n다른 곳에 있죠 사랑한다는\n그 말 한마디 못한 채\n...
4,사랑연가,언타이틀,1999.02.01,댄스,유건형\n작사\n유건형\n작곡\n유건형\n편곡,누굴 좋아해야 하나요\n누굴 사랑해야 하나요\n누굴 향해 고백하나요\n난 정말 알 ...


In [18]:
test_p.columns = column_names
test_p.tail(350)

,title,artist,release_date,song_genre,creator,lyrics
0,떠나가지 마세요,언타이틀,1999.08.01,댄스,,
1,학교 (Main Title),언타이틀,1999.03.05,-,,다시 바꿔 생각해 너무 억울한걸\n다른 위치라면은 누구나 같은데\n학교폭력에 숨진 ...
2,떠나가지 마세요,언타이틀,1999.02.01,댄스,유건형\n작사\n서정환\n작사\n유건형\n작곡\n유건형\n편곡,난 울고 있어요 그대도 아시나요\n미치도록 아픈 나의 가슴을\n그대 날 떠난 후엔 ...
3,나의 슬픈 사랑이야기,언타이틀,1999.02.01,댄스,유건형\n작사\n유건형\n작곡\n유건형\n편곡,이젠 늦었어요 이미 그녀는\n다른 곳에 있죠 사랑한다는\n그 말 한마디 못한 채\n...
4,사랑연가,언타이틀,1999.02.01,댄스,유건형\n작사\n유건형\n작곡\n유건형\n편곡,누굴 좋아해야 하나요\n누굴 사랑해야 하나요\n누굴 향해 고백하나요\n난 정말 알 ...
...,...,...,...,...,...,...
81,진달래꽃,오피피에이,1998.06,댄스,,나 기도했어 너의 모습 보면서\n정해진 우리의 시간\n멈추길 기도해\n지켜주겠다던 ...
82,None,None,1998.06,None,None,None
83,None,None,1998.06,None,None,None
84,None,None,1998.06,None,None,None


In [19]:
test_p.shape

(86, 6)

In [20]:
# 곡 전처리
test_p['title'].isnull().sum()

4

In [21]:
test_p.tail()

,title,artist,release_date,song_genre,creator,lyrics
81,진달래꽃,오피피에이,1998.06,댄스,,나 기도했어 너의 모습 보면서\n정해진 우리의 시간\n멈추길 기도해\n지켜주겠다던 ...
82,None,None,1998.06,None,None,None
83,None,None,1998.06,None,None,None
84,None,None,1998.06,None,None,None
85,None,None,,None,None,None


In [22]:
test_p.to_csv('k_pop_3.tsv',index=False,header=None,sep='\t') #tsv파일로 저장